In [2]:
import os, sys
import numpy as np
import itertools
from collections import defaultdict

from tqdm.notebook import trange
from tqdm import tqdm

In [3]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

In [4]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

In [5]:
# load functions from another notebook
%run helpers.ipynb

In [6]:
pooling_model = load_model('pooling')

### This code is designed to generate and score different item combinations for a list of target users based on their past interactions and the recommendations provided by a recommendation model. It stores the scores and combinations in the materialized_pred_scores dictionary for further analysis and evaluation of recommendation performance.

In [7]:
target_item_pos = [3]

tuples_length = 3
top_k = 10
model = pooling_model
num_users = 50
materialized_pred_scores = defaultdict(list)

with tqdm(total=len(target_item_pos), desc='target position loop') as pbar:
    for pos in target_item_pos:
        pbar.update(10)

        for user_id in trange(1, num_users, desc='users loop', leave=False):
            seq_size = len(test.sequences[test.user_ids == user_id]) # For each user, a sequence length (seq_size) is calculated based on their interactions.

            for j in range(seq_size):
                if all(v > 0 for v in test.sequences[test.user_ids == user_id][j]):
                    items_interacted = test.sequences[test.user_ids == user_id][j]

                    predictions = -model.predict(items_interacted) # the - is used to reverse the order of predictions, because the recommendation model outputs higher values for items that are more likely to be preferred by the user
                    predictions[items_interacted] = StaticVars.FLOAT_MAX # This line sets the prediction score for items the user has already interacted with to a very high value. This step is likely used to ensure that these items are not recommended again, as they are already known interactions.

                    target_item = predictions.argsort()[min(top_k, int(pos)) - 1]

                    for r in range(tuples_length):
                        combs = itertools.combinations(items_interacted, r + 1)
                        for c in combs:
                            preds = -model.predict(list(set(items_interacted).difference(set(c))))
                            preds[items_interacted] = StaticVars.FLOAT_MAX

                            materialized_pred_scores[user_id].append([c, preds[target_item], preds[preds.argsort()[top_k - 1]]])
                            

target position loop:   0%|          | 0/1 [00:00<?, ?it/s]

users loop:   0%|          | 0/49 [00:00<?, ?it/s]

target position loop: 10it [01:27,  8.73s/it]              


In [8]:
print(materialized_pred_scores)
after_top_k = []
before_top_k = []
for k, val in materialized_pred_scores.items():
    for comb in val:
        if (comb[1] / comb[2]) < 1:
            before_top_k.append(k)
        else: after_top_k.append(k)

defaultdict(<class 'list'>, {5: [[(230,), -3.4583127, -3.0922828], [(257,), -3.508264, -3.1879864], [(359,), -3.3235564, -2.9861364], [(130,), -3.4910882, -3.1077282], [(358,), -3.3457215, -3.0167298], [(329,), -3.4062405, -3.0301182], [(372,), -3.3377304, -3.0031638], [(227,), -3.4033232, -3.0485673], [(324,), -3.527669, -3.1466131], [(301,), -3.4700117, -3.134161], [(253,), -3.3830035, -3.0872185], [(239,), -3.3321671, -2.9925892], [(107,), -3.3743227, -3.0529866], [(305,), -3.5539, -3.1927907], [(266,), -3.313138, -2.9802136], [(315,), -3.3253691, -2.9857552], [(60,), -3.3917167, -3.0444682], [(331,), -3.3841689, -3.0600402], [(123,), -3.4404528, -3.1257868], [(357,), -3.4645953, -3.1339812], [(230, 257), -3.5547872, -3.207817], [(230, 359), -3.3603578, -3.026141], [(230, 130), -3.5367079, -3.1563344], [(230, 358), -3.38369, -3.027547], [(230, 329), -3.4473941, -3.031467], [(230, 372), -3.3752782, -3.013267], [(230, 227), -3.4443235, -3.0481892], [(230, 324), -3.5752137, -3.213657],

: 